In [5]:
import sys
import os
from argparse import ArgumentParser
import cv2
import numpy as np
import logging as log
from openvino.inference_engine import IECore
import matplotlib.pyplot as plt
import time

In [4]:

# logging
log.basicConfig(format="[ %(levelname)s ] %(message)s",
                level=log.INFO, stream=sys.stdout)

log.info("creating inference engine")
ie = IECore()

log.info("Loading network")
# net = ie.read_network(args.model, os.path.splitext(args.model)[0] + ".bin")
net = ie.read_network('model-small.onnx')

assert len(net.input_info) == 1, "Sample supports only single input topologies"
assert len(net.outputs) == 1, "Sample supports only single output topologies"

log.info("preparing input blobs")
input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))
net.batch_size = 1

# read and pre-process input image
_, _, height, width = net.input_info[input_blob].input_data.shape

input_image = '256x256_ryan.png'

image = cv2.imread(input_image, cv2.IMREAD_COLOR)
(input_height, input_width) = image.shape[:-1]

# resize
if (input_height, input_width) != (height, width):
    log.info("Image is resized from {} to {}".format(
        image.shape[:-1], (height, width)))
    image = cv2.resize(image, (width, height), cv2.INTER_CUBIC)

# prepare input
image = image.astype(np.float32)
image = image.transpose((2, 0, 1))
image_input = np.expand_dims(image, 0)

# loading model to the plugin
log.info("loading model to the plugin")
exec_net = ie.load_network(network=net, device_name='CPU')

start = time.time()
print("start")

# start sync inference
log.info("starting inference")
res = exec_net.infer(inputs={input_blob: image_input})

# processing output blob
log.info("processing output blob")
disp = np.squeeze(res[out_blob][0])

end = time.time()
print("end")
print(end - start)

# resize disp to input resolution
disp = cv2.resize(disp, (input_width, input_height), cv2.INTER_CUBIC)

# rescale disp
disp_min = disp.min()
disp_max = disp.max()

if disp_max - disp_min > 1e-6:
    disp = (disp - disp_min) / (disp_max - disp_min)
else:
    disp.fill(0.5)

# pfm
#     out = 'disp.pfm'
#     cv2.imwrite(out, disp)

#     log.info("Disparity map was saved to {}".format(out))

# png
out = f'{input_image}_mono.png'
plt.imsave(out, disp, vmin=0, vmax=1, cmap='inferno')

log.info("Color-coded disparity image was saved to {}".format(out))

[ INFO ] creating inference engine
[ INFO ] Loading network
[ INFO ] preparing input blobs
[ INFO ] loading model to the plugin
start
[ INFO ] starting inference
[ INFO ] processing output blob
end
0.04198932647705078
[ INFO ] Color-coded disparity image was saved to 256x256_ryan.png_mono.png


In [13]:
# Open the video
cap = cv2.VideoCapture('mono_in_hollis.mp4')

# Initialize frame counter
cnt = 0

# Some characteristics from the original video
w_frame, h_frame = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps, frames = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Here you can define your croping values
x,y,h,w = 0,0,256,256

# output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('result.avi', fourcc, fps, (w, h))


# Now we start
while(cap.isOpened()):
    ret, frame = cap.read()

    cnt += 1 # Counting frames

    # Avoid problems when video finish
    if ret==True:
        # Croping the frame
        crop_frame = frame[y:y+h, x:x+w]

        # Percentage
        xx = cnt *100/frames
        print(int(xx),'%')

        # Saving from the desired frames
        #if 15 <= cnt <= 90:
        #    out.write(crop_frame)

        # I see the answer now. Here you save all the video
        out.write(crop_frame)

        # Just to see the video in real time          
        cv2.imshow('frame',frame)
        cv2.imshow('croped',crop_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break


cap.release()
out.release()
cv2.destroyAllWindows()

0 %


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-memyuvq3\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
!ffmpeg -i mono_in_hollis.mp4 -vf "crop=256:256:100:100" -c:a copy out.mp4